# SLC45A2
[Albinism type IV (OCA4)](https://omim.org/entry/606574) is caused by homozygous or compound heterozygous mutation SLC45A2.

In [1]:
import gpsea
import hpotk

store = hpotk.configure_ontology_store()
hpo = store.load_minimal_hpo(release="v2025-01-16")
print(f'Loaded HPO v{hpo.version}')
print(f"Using genophenocorr version {gpsea.__version__}")

Loaded HPO v2025-01-16
Using genophenocorr version 0.9.6.dev0


# SLC45A2
We user the [Matched Annotation from NCBI and EMBL-EBI (MANE)](https://www.ncbi.nlm.nih.gov/refseq/MANE/) transcript and the corresponding protein identifier for SLC45A2.

In [2]:
gene_symbol = 'SLC45A2'
mane_tx_id = 'NM_016180.5'
mane_protein_id = "NP_057264.4" #membrane-associated transporter protein isoform


In [3]:
from ppktstore.registry import configure_phenopacket_registry
from gpsea.preprocessing import configure_caching_cohort_creator, load_phenopackets

from ppktstore.registry import configure_phenopacket_registry
phenopacket_registry = configure_phenopacket_registry()
with phenopacket_registry.open_phenopacket_store("0.1.24") as ps:
    phenopackets = tuple(ps.iter_cohort_phenopackets(gene_symbol))

cohort_creator = configure_caching_cohort_creator(hpo)
cohort, validation = load_phenopackets(
    phenopackets=phenopackets, 
    cohort_creator=cohort_creator,
)
validation.summarize()

Individuals Processed: 100%|██████████| 30/30 [00:00<00:00, 35.29 individuals/s]
Validated under permissive policy
No errors or warnings were found


In [4]:
from gpsea.view import CohortViewer
cv = CohortViewer(hpo=hpo)
cv.process(cohort=cohort, transcript_id=mane_tx_id)

n,HPO Term
30,Hypopigmentation of hair
29,White eyelashes
29,White eyebrow
28,Nystagmus
28,Iris hypopigmentation
26,Reduced visual acuity
26,Hypoplasia of the fovea
24,Iris transillumination defect
12,Melanocytic nevus
10,Nevus


In [5]:
# Note we do not show the protein structure because no domains/motifs are shown in UniProt (Q9UMX9).
from gpsea.view import CohortVariantViewer

viewer = CohortVariantViewer(tx_id=mane_tx_id)
viewer.process(cohort)

Count,Variant key,HGVS,Overlapping Exons,Effects
12,5_33954406_33954407_TG_T,c.986del (p.Thr329LysfsTer69),4,frameshift
4,5_33944770_33944770_C_T,c.1471G>A (p.Gly491Arg),7,missense
3,5_33947363_33947365_CTT_C,c.1166_1167del (p.Lys389SerfsTer55),6,frameshift
2,5_33951674_33951674_C_A,c.1036G>T (p.Val346Leu),5,missense
2,5_33944709_33944709_G_T,c.1532C>A (p.Ala511Glu),7,missense
2,5_33984325_33984326_CG_C,c.258del (p.Val87TrpfsTer26),1,frameshift
2,5_33947257_33947258_AG_A,c.1273del (p.Leu425TrpfsTer9),6,frameshift
2,5_33954443_33954443_T_C,c.950A>G (p.Tyr317Cys),4,missense
2,5_33944734_33944735_TC_T,c.1506del (p.Thr503ProfsTer6),7,frameshift
2,5_33954363_33954365_GGC_G,c.1028_1029del (p.Gly343AlafsTer10),4,frameshift


# Genotype-Phenotype Correlation (GPC) Analysis

In [6]:
from gpsea.analysis.pcats import configure_hpo_term_analysis
from gpsea.analysis.clf import prepare_classifiers_for_terms_of_interest

analysis = configure_hpo_term_analysis(hpo)
pheno_clfs = prepare_classifiers_for_terms_of_interest(
    cohort=cohort,
    hpo=hpo,
)

In [7]:
from gpsea.model import VariantEffect
from gpsea.analysis.predicate import variant_effect
from gpsea.analysis.clf import biallelic_classifier
from gpsea.view import MtcStatsViewer

missense = variant_effect(VariantEffect.MISSENSE_VARIANT, tx_id=mane_tx_id)
missense_clf = biallelic_classifier(
    a_predicate=missense, 
    b_predicate=~missense, 
    a_label="missense",
    b_label="other",
    partitions=((0,1),(2,))
)
missense_result = analysis.compare_genotype_vs_phenotypes(
    cohort=cohort,
    gt_clf=missense_clf,
    pheno_clfs=pheno_clfs,
)

viewer = MtcStatsViewer()
viewer.process(missense_result)

Reason,Count
Skip terms if all counts are identical to counts for a child term,14
"Skipping ""general"" level terms",19
Skipping terms that are rare on the cohort level (in less than 40% of the cohort members),5


In [8]:
from gpsea.view import summarize_hpo_analysis

summarize_hpo_analysis(hpo=hpo, result=missense_result)

Allele group,missense/missense OR missense/other,other/other,Corrected p values,p values
Iris transillumination defect [HP:0012805],13/17 (76%),11/11 (100%),1.0,0.132357
Esotropia [HP:0000565],4/16 (25%),5/10 (50%),1.0,0.234117
Heterotropia [HP:0032012],6/18 (33%),7/12 (58%),1.0,0.264122
Strabismus [HP:0000486],6/18 (33%),7/12 (58%),1.0,0.264122
Nystagmus [HP:0000639],16/18 (89%),12/12 (100%),1.0,0.503448
Iris hypopigmentation [HP:0007730],16/18 (89%),12/12 (100%),1.0,0.503448
Exotropia [HP:0000577],2/14 (14%),2/7 (29%),1.0,0.574269
Melanocytic nevus [HP:0000995],9/14 (64%),3/6 (50%),1.0,0.642415
Localized skin lesion [HP:0011355],13/13 (100%),9/9 (100%),1.0,1.000000
Nevus [HP:0003764],13/18 (72%),9/12 (75%),1.0,1.000000


In [9]:
from gpsea.analysis.clf import sex_classifier
mf_result = analysis.compare_genotype_vs_phenotypes(
    cohort=cohort,
    gt_clf=sex_classifier(),
    pheno_clfs=pheno_clfs,
)
summarize_hpo_analysis(hpo=hpo, result=mf_result)

Sex,FEMALE,MALE,Corrected p values,p values
White eyebrow [HP:0002226],17/17 (100%),12/13 (92%),1.0,0.433333
Melanocytic nevus [HP:0000995],5/10 (50%),7/10 (70%),1.0,0.649917
Esotropia [HP:0000565],4/14 (29%),5/12 (42%),1.0,0.682838
Localized skin lesion [HP:0011355],12/12 (100%),10/10 (100%),1.0,1.000000
Nevus [HP:0003764],12/17 (71%),10/13 (77%),1.0,1.000000
Iris transillumination defect [HP:0012805],14/17 (82%),10/11 (91%),1.0,1.000000
Reduced visual acuity [HP:0007663],15/15 (100%),11/11 (100%),1.0,1.000000
Hypoplasia of the fovea [HP:0007750],16/16 (100%),10/10 (100%),1.0,1.000000
Abnormal fundus morphology [HP:0001098],16/16 (100%),10/10 (100%),1.0,1.000000
Abnormal retinal morphology [HP:0000479],16/16 (100%),10/10 (100%),1.0,1.000000


# Summary

In [10]:
from gpseacs.report import GpseaAnalysisReport, GPAnalysisResultSummary

fet_results = (
    GPAnalysisResultSummary.from_multi(
        result=missense_result,
    ),
    GPAnalysisResultSummary.from_multi(
        result=mf_result,
    ),
)

caption = "No significant association identified."
report = GpseaAnalysisReport(name=gene_symbol, 
                             cohort=cohort, 
                             fet_results=fet_results,
                             gene_symbol=gene_symbol,
                             mane_tx_id=mane_tx_id,
                             mane_protein_id=mane_protein_id,
                             caption=caption)

In [11]:
from gpseacs.report import GpseaNotebookSummarizer
summarizer = GpseaNotebookSummarizer(hpo=hpo, gpsea_version=gpsea.__version__)
summarizer.summarize_report(report=report)

Genotype (A),Genotype (B),Tests performed,Significant tests
missense/missense OR missense/other,other/other,25,0
Genotype (A),Genotype (B),Tests performed,Significant tests
FEMALE,MALE,25,0


In [12]:
summarizer.process_latex(report=report)

Output to ../../supplement/tex/SLC45A2_summary_draft.tex
